In [1]:
import pandas as pd
import os

In [3]:
factor_name = 'close_ret'

fac_dirpath = f'/home/wangzirui/workspace/data/param_scan/{factor_name}'

dfs = []
for date in pd.date_range('2023.09.22', '2023.09.30'):
    date = date.strftime('%Y.%m.%d')
    fac_filepath = f'{fac_dirpath}/{date}.pkl'
    if os.path.exists(fac_filepath):
        df = pd.read_pickle(fac_filepath)
        dfs.append(df)


In [11]:
res = pd.concat([df['metric'] for df in dfs], axis=1)

In [12]:
res.T.mean()

0    0.112884
1    0.154514
2    0.166428
3    0.170831
4    0.151840
5    0.134178
dtype: float64

In [13]:
df_tmp = dfs[0].copy()
df_tmp['metric'] = res.T.mean()

In [14]:
df_tmp

,shift1,metric
0,1,0.112884
1,5,0.154514
2,10,0.166428
3,20,0.170831
4,60,0.151840
5,100,0.134178


In [50]:
fac_filepath = "/home/wangzirui/workspace/data/param_optimized/fac_2023.09.22.pkl"
fac_df = pd.read_pickle(fac_filepath)

In [55]:
ret_filepath = "/home/wangzirui/workspace/data/close_return/2023.09.22.pkl"
ret_df = pd.read_pickle(ret_filepath)

In [56]:
ret_df = ret_df.reset_index()
ret_df.head(1)

,tradetime,securityid,1m,3m,5m
0,2023-09-22 09:30:00,000001.SZ,NaN,NaN,NaN


In [69]:
def get_corr(fac_df, ret_df):
    fac_arr = fac_df.pivot(index='tradetime', columns='securityid', values='factor')
    res = []
    common_index = None
    common_col = None
    fac_com = None
    for idx, i in enumerate(['1m', '3m', '5m']):
        ret_arr = ret_df.pivot(index='tradetime', columns='securityid', values=i)
        if (idx==0):
            common_index = fac_arr.index.intersection(ret_arr.index)
            common_col = fac_arr.columns.intersection(ret_arr.columns)
            fac_com = fac_arr.loc[common_index, common_col]
        
        ret_com = ret_arr.loc[common_index, common_col]
        tmp = fac_com.corrwith(ret_com)
        tmp.name = i
        res.append(tmp)
        
    return pd.concat(res, axis=1)

In [70]:
factor_and_ret = fac_df.merge(ret_df, how='left', on=['tradetime', 'securityid'], sort=True)
corr = get_corr(fac_df, ret_df)

In [71]:
corr

,1m,3m,5m
securityid,,,
000001.SZ,-0.325920,-0.243234,-0.217098
000002.SZ,-0.264176,-0.185927,-0.197140
000004.SZ,-0.126901,-0.096989,-0.057588
000005.SZ,-0.313582,-0.278675,-0.254014
000006.SZ,-0.332676,-0.273949,-0.233139
...,...,...,...
688799.SH,-0.160479,-0.085675,-0.061032
688800.SH,-0.125503,-0.078491,-0.072826
688819.SH,-0.109607,-0.094547,-0.100335


In [63]:
corr.name='1m'
corr

securityid
000001.SZ   -0.325920
000002.SZ   -0.264176
000004.SZ   -0.126901
000005.SZ   -0.313582
000006.SZ   -0.332676
               ...   
688799.SH   -0.160479
688800.SH   -0.125503
688819.SH   -0.109607
688981.SH   -0.075474
689009.SH   -0.006665
Name: 1m, Length: 5059, dtype: float64

In [60]:
import sys
sys.path.insert(0, '../')
from factor_cal.factor_eval.basic_evaluate import factor_timeSeries_information_coefficient
fac_df.rename(columns={'close_ret': 'factor'}, inplace=True)
factor_and_ret = fac_df.merge(ret_df, how='left', on=['tradetime', 'securityid'], sort=True)
factor_and_ret = factor_and_ret.dropna()
    
ic_data = factor_timeSeries_information_coefficient(factor_and_ret)

/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/wangzirui/miniconda3/envs/py38/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/wangzirui/minicond

In [61]:
ic_data

,1m,3m,5m
securityid,,,
000001.SZ,-0.072110,-0.066854,0.005320
000002.SZ,-0.144325,-0.090767,-0.092485
000004.SZ,0.003477,-0.010861,-0.019298
000005.SZ,0.341841,0.103007,-0.778537
000006.SZ,-0.270585,-0.176581,-0.177935
...,...,...,...
688799.SH,-0.129733,-0.176890,-0.205560
688800.SH,-0.066269,-0.113079,-0.064010
688819.SH,-0.092099,-0.040971,-0.024835


In [ ]:
import numpy as np
def get_corr2(factor, ret):
    factor.set_index(['tradetime', 'securityid'], inplace=True)
    # factor.rename(columns={'value': 'factor'}, inplace=True)
    # factor.replace(np.inf, np.nan, inplace=True)
    
    factor_and_ret = factor.merge(ret, how='left', on=['tradetime', 'securityid'], sort=True)
    factor_and_ret = factor_and_ret.dropna()

In [46]:
fac_df.set_index(['tradetime', 'securityid'], inplace=True)

In [ ]:
fac_df.drop

In [48]:
factor_and_ret = fac_df.merge(ret_df, how='left', on=['tradetime', 'securityid'], sort=True)

In [38]:
arr1 = fac_com.to_numpy()
arr2 = ret_com.to_numpy()